# Seq2Seq LLM Reprompt




In [1]:
from datasets import load_dataset
import pandas as pd 

# df.columns
ds = load_dataset("csv",data_files={"train":'gemma1000_w7b.csv'})

/home/user/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df=df[['original_text','gemma_7b_rewritten_text_temp0','rewrite_prompt']]
# df['diff_prompt']='Tell me what prompt converted the following text: \n"""' + df['original_text'] + '"""\n to this text \n"""' + df['gemma_7b_rewritten_text_temp0'] + '"""\n'
#original text prefix

orig_prefix = "Original Text:"

rewrite_prefix = "\nRewritten Text:"

import json

def get_prompt(orig_text, transformed_text):
    message = f"{orig_prefix} {orig_text} {rewrite_prefix} {transformed_text}"
    return message


# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for item in ds["train"]:
            json_object = {
                "text": get_prompt(item["original_text"],item['gemma_7b_rewritten_text_temp0']),
                "output": item['rewrite_prompt']
            }
            output_jsonl_file.write(json.dumps(json_object) + "\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./training_dataset.jsonl")


In [3]:
from datasets import Dataset
train_dataset = load_dataset('json', data_files='./training_dataset.jsonl', split="train")

train_dataset=train_dataset.train_test_split(test_size=0.1)


Generating train split: 1000 examples [00:00, 93933.17 examples/s]


In [4]:
model_checkpoint = "t5-base"

In [5]:
from datasets import load_dataset
from evaluate import load

metric = load("rouge")

2024-03-21 01:30:26.106833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 01:30:26.106877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 01:30:26.107909: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 01:30:26.112957: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 01:30:26.750714: W tensorflow/compiler/tf2

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device_map="auto")

In [7]:
max_input_length = 1024
max_target_length = 8

def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["output"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_dataset = train_dataset.map(preprocess_function, batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4774.34 examples/s]


In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 900
    })
    test: Dataset({
        features: ['text', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [10]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, device_map="auto")

In [11]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-reprompt",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True
)

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [14]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],    
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/user/envs/test_env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.037107,66.273900,53.007400,66.370000,66.164300,8.050000
2,No log,0.774651,66.890100,53.205800,66.889100,66.744000,7.730000
3,1.496800,0.706097,68.997900,53.499200,68.839900,68.760900,7.900000
4,1.496800,0.683976,69.325100,55.706000,69.326100,69.107600,7.780000
5,0.701800,0.647471,69.273000,55.113600,69.198000,69.052700,7.930000
6,0.701800,0.629148,68.802900,55.296500,68.742700,68.571400,7.980000
7,0.574300,0.615443,68.161000,54.452000,68.155600,67.992200,7.960000
8,0.574300,0.602271,68.629600,55.265900,68.575500,68.429900,7.990000
9,0.522700,0.595636,69.776700,56.908800,69.575500,69.453800,7.970000
10,0.522700,0.592194,69.666400,56.747500,69.494200,69.401100,7.990000


/home/user/envs/test_env/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/user/envs/test_env/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Checkpoint destination directory t5-base-finetuned-reprompt/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/user/envs/test_env/lib/python3.10/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the 

TrainOutput(global_step=2250, training_loss=0.7866869676378038, metrics={'train_runtime': 419.0865, 'train_samples_per_second': 21.475, 'train_steps_per_second': 5.369, 'total_flos': 5956269459886080.0, 'train_loss': 0.7866869676378038, 'epoch': 10.0})

In [16]:
trainer.save_model('Seq2Seq_tst')

In [17]:
#tokenizer = AutoTokenizer.from_pretrained('./Seq2Seq_tst', device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained('./Seq2Seq_tst', device_map="auto")

In [18]:
# PRediction
from transformers import pipeline
pipe=pipeline(task="summarization", model=model, tokenizer=tokenizer, max_length=200)
pipe(train_dataset["train"][0]["text"])

[{'summary_text': "Explain this to me like I've listened to a rap . Explain this as if I were a singer ."}]

In [19]:
train_dataset["train"][0]["output"]

'Turn this into a jazz song’s scat lyrics.'

In [22]:
print(pipe(train_dataset["train"][10]["text"]))
print(train_dataset["train"][10]["output"])

[{'summary_text': 'Explain this as if it were a magic . the outcome of this case will be a testament to the power and potential abuse .'}]
Explain this as if it were a magic trick.
